In [14]:
import os
import re
import sys
import time
import unicodedata

import numpy as np
import pandas as pd
import yfinance as yf

try:
    __file__
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
except NameError:
    from pathlib import Path
    Path().resolve()
    sys.path.append(os.path.join(Path().resolve(), '..'))

from structs.res import AppRes
from funcs.conv import df_to_html

In [2]:
# JPX ticker list
res = AppRes()
df_tse = res.getJPXTickerList()
df_tse

,日付,コード,銘柄名,市場・商品区分,33業種コード,33業種区分,17業種コード,17業種区分,規模コード,規模区分
0,20250131,1301,極洋,プライム（内国株式）,50,水産・農林業,1,食品,7,TOPIX Small 2
1,20250131,1305,ｉＦｒｅｅＥＴＦ ＴＯＰＩＸ（年１回決算型）,ETF・ETN,-,-,-,-,-,-
2,20250131,1306,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,ETF・ETN,-,-,-,-,-,-
3,20250131,1308,上場インデックスファンドＴＯＰＩＸ,ETF・ETN,-,-,-,-,-,-
4,20250131,1309,ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,ETF・ETN,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...
4405,20250131,9991,ジェコス,プライム（内国株式）,6050,卸売業,13,商社・卸売,7,TOPIX Small 2
4406,20250131,9993,ヤマザワ,スタンダード（内国株式）,6100,小売業,14,小売,-,-
4407,20250131,9994,やまや,スタンダード（内国株式）,6100,小売業,14,小売,-,-
4408,20250131,9996,サトー商会,スタンダード（内国株式）,6050,卸売業,13,商社・卸売,-,-


In [3]:
df_tse.columns

Index(['日付', 'コード', '銘柄名', '市場・商品区分', '33業種コード', '33業種区分', '17業種コード', '17業種区分',
       '規模コード', '規模区分'],
      dtype='object')

In [4]:
np.sort(df_tse['市場・商品区分'].unique())

array(['ETF・ETN', 'PRO Market', 'REIT・ベンチャーファンド・カントリーファンド・インフラファンド',
       'グロース（内国株式）', 'グロース（外国株式）', 'スタンダード（内国株式）', 'スタンダード（外国株式）',
       'プライム（内国株式）', 'プライム（外国株式）', '出資証券'], dtype=object)

In [5]:
market_target = ['グロース（内国株式）', 'スタンダード（内国株式）', 'プライム（内国株式）']

In [6]:
df_target = df_tse[df_tse['市場・商品区分'].isin(market_target)].reset_index(drop=True)
df_target['コード'] = df_target['コード'].astype(str)
df_target['日時'] = None
df_target['日時'] = df_target['日時'].astype(str)
df_target['高値'] = np.nan
df_target['安値'] = np.nan
df_target['変化率'] = np.nan
df_target['出来高'] = np.nan
df_target['増減'] = None
df_target['増減'] = df_target['増減'].astype(str)
df_target

,日付,コード,銘柄名,市場・商品区分,33業種コード,33業種区分,17業種コード,17業種区分,規模コード,規模区分,日時,高値,安値,変化率,出来高,増減
0,20250131,1301,極洋,プライム（内国株式）,50,水産・農林業,1,食品,7,TOPIX Small 2,None,NaN,NaN,NaN,NaN,None
1,20250131,130A,Ｖｅｒｉｔａｓ Ｉｎ Ｓｉｌｉｃｏ,グロース（内国株式）,3250,医薬品,5,医薬品,-,-,None,NaN,NaN,NaN,NaN,None
2,20250131,1332,ニッスイ,プライム（内国株式）,50,水産・農林業,1,食品,4,TOPIX Mid400,None,NaN,NaN,NaN,NaN,None
3,20250131,1333,マルハニチロ,プライム（内国株式）,50,水産・農林業,1,食品,4,TOPIX Mid400,None,NaN,NaN,NaN,NaN,None
4,20250131,135A,ＶＲＡＩＮ Ｓｏｌｕｔｉｏｎ,グロース（内国株式）,5250,情報・通信業,10,情報通信・サービスその他,-,-,None,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3824,20250131,9991,ジェコス,プライム（内国株式）,6050,卸売業,13,商社・卸売,7,TOPIX Small 2,None,NaN,NaN,NaN,NaN,None
3825,20250131,9993,ヤマザワ,スタンダード（内国株式）,6100,小売業,14,小売,-,-,None,NaN,NaN,NaN,NaN,None
3826,20250131,9994,やまや,スタンダード（内国株式）,6100,小売業,14,小売,-,-,None,NaN,NaN,NaN,NaN,None
3827,20250131,9996,サトー商会,スタンダード（内国株式）,6050,卸売業,13,商社・卸売,-,-,None,NaN,NaN,NaN,NaN,None


In [7]:
for r in range(len(df_target)):
    code = df_target.at[r, 'コード']
    symbol = '%s.T' % code
    ticker = yf.Ticker(symbol)
    try:
        df = ticker.history(period='1d')
    except Exception as e:
        print(e)
        time.sleep(60)
        df = ticker.history(period='1d')
        
    if len(df) > 0:
        v_open = df['Open'].iloc[0]
        v_high = df['High'].iloc[0]
        v_low = df['Low'].iloc[0]
        v_close = df['Close'].iloc[0]
        v_volume = df['Volume'].iloc[0]

        df_target.at[r, '高値'] = v_high
        df_target.at[r, '安値'] = v_low
        df_target.at[r, '変化率'] = (v_high - v_low) * 2 / (v_high + v_low)
        df_target.at[r, '出来高'] = v_volume
        df_target.at[r, '日時'] = df.index[0]

        if v_open > v_close:
            df_target.at[r, '増減'] = '▼'
        elif v_open < v_close:
            df_target.at[r, '増減'] = '△'
        else:
            df_target.at[r, '増減'] = '-'

df_target

$1439.T: possibly delisted; no price data found  (period=1d)
Could not get exchangeTimezoneName for ticker '1968.T' reason: 'chart'
$1968.T: possibly delisted; no price data found  (period=1d)
Could not get exchangeTimezoneName for ticker '2157.T' reason: 'chart'
$2157.T: possibly delisted; no price data found  (period=1d)
$3322.T: possibly delisted; no price data found  (period=1d)
Could not get exchangeTimezoneName for ticker '3561.T' reason: 'chart'
$3561.T: possibly delisted; no price data found  (period=1d)
$3990.T: possibly delisted; no price data found  (period=1d)
Could not get exchangeTimezoneName for ticker '4372.T' reason: 'chart'
$4372.T: possibly delisted; no price data found  (period=1d)
Could not get exchangeTimezoneName for ticker '4816.T' reason: 'chart'
$4816.T: possibly delisted; no price data found  (period=1d)
$6164.T: possibly delisted; no price data found  (period=1d)


Too Many Requests. Rate limited. Try after a while.


Could not get exchangeTimezoneName for ticker '6845.T' reason: 'chart'
$6845.T: possibly delisted; no price data found  (period=1d)
Could not get exchangeTimezoneName for ticker '6947.T' reason: 'chart'
$6947.T: possibly delisted; no price data found  (period=1d)


Too Many Requests. Rate limited. Try after a while.


Could not get exchangeTimezoneName for ticker '8113.T' reason: 'chart'
$8113.T: possibly delisted; no price data found  (period=1d)
$8732.T: possibly delisted; no price data found  (period=1d)
Could not get exchangeTimezoneName for ticker '8841.T' reason: 'chart'
$8841.T: possibly delisted; no price data found  (period=1d)
Could not get exchangeTimezoneName for ticker '9037.T' reason: 'chart'
$9037.T: possibly delisted; no price data found  (period=1d)
Could not get exchangeTimezoneName for ticker '9404.T' reason: 'chart'
$9404.T: possibly delisted; no price data found  (period=1d)
$94345.T: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$9675.T: possibly delisted; no price data found  (period=1d)
Could not get exchangeTimezoneName for ticker '9757.T' reason: 'chart'
$9757.T: possibly delisted; no price data found  (period=1d)


,日付,コード,銘柄名,市場・商品区分,33業種コード,33業種区分,17業種コード,17業種区分,規模コード,規模区分,日時,高値,安値,変化率,出来高,増減
0,20250131,1301,極洋,プライム（内国株式）,50,水産・農林業,1,食品,7,TOPIX Small 2,2025-02-21 00:00:00+09:00,4125.0,4080.000000,0.010969,40100.0,△
1,20250131,130A,Ｖｅｒｉｔａｓ Ｉｎ Ｓｉｌｉｃｏ,グロース（内国株式）,3250,医薬品,5,医薬品,-,-,2025-02-21 00:00:00+09:00,867.0,836.000000,0.036406,23500.0,▼
2,20250131,1332,ニッスイ,プライム（内国株式）,50,水産・農林業,1,食品,4,TOPIX Mid400,2025-02-21 00:00:00+09:00,860.5,851.599976,0.010397,858800.0,▼
3,20250131,1333,マルハニチロ,プライム（内国株式）,50,水産・農林業,1,食品,4,TOPIX Mid400,2025-02-21 00:00:00+09:00,3178.0,3117.000000,0.019380,184400.0,△
4,20250131,135A,ＶＲＡＩＮ Ｓｏｌｕｔｉｏｎ,グロース（内国株式）,5250,情報・通信業,10,情報通信・サービスその他,-,-,2025-02-21 00:00:00+09:00,2468.0,2293.000000,0.073514,61900.0,△
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3824,20250131,9991,ジェコス,プライム（内国株式）,6050,卸売業,13,商社・卸売,7,TOPIX Small 2,2025-02-21 00:00:00+09:00,1023.0,1011.000000,0.011799,40500.0,△
3825,20250131,9993,ヤマザワ,スタンダード（内国株式）,6100,小売業,14,小売,-,-,2025-02-21 00:00:00+09:00,1155.0,1149.000000,0.005208,4000.0,△
3826,20250131,9994,やまや,スタンダード（内国株式）,6100,小売業,14,小売,-,-,2025-02-21 00:00:00+09:00,2894.0,2880.000000,0.004849,3400.0,△
3827,20250131,9996,サトー商会,スタンダード（内国株式）,6050,卸売業,13,商社・卸売,-,-,2025-02-21 00:00:00+09:00,1920.0,1892.000000,0.014690,2000.0,△


In [19]:
# 「日時」列が 'None' の行を除外
df_target = df_target[df_target['日時'] != 'None'].copy()

# ユニークな日時を抽出
dates = df_target['日時'].unique()
# 万が一複数の日時が存在した場合、一番新しい日時のデータのみ扱う
dt = sorted(dates, reverse=True)
df_target = df_target[df_target['日時'] == dt[0]].copy()

# 「変化率」列で逆ソート
df_target = df_target.sort_values('変化率', ascending=False).reset_index(drop=True)

# 順位が判るようにインデックス列から「#」列を作成
df_target['#'] = df_target.index + 1

# 「銘柄名」列の文字列を規格化
df_target['銘柄名'] = [unicodedata.normalize('NFKC', s) for s in df_target['銘柄名']]

# 「市場・商品区分」列の '（内国株式）' を除外
pattern = re.compile(r'(.+)（内国株式）')
list_s = list()
for s in df_target['市場・商品区分']:
    m = pattern.match(s)
    if m:
        list_s.append(m.group(1))
    else:
        list_s.append(m)
df_target['市場・商品区分'] = list_s

df_target

,日付,コード,銘柄名,市場・商品区分,33業種コード,33業種区分,17業種コード,17業種区分,規模コード,規模区分,日時,高値,安値,変化率,出来高,増減,#
0,20250131,3559,ピーバンドットコム,None,6050,卸売業,13,商社・卸売,-,-,2025-02-21 00:00:00+09:00,899.0,633.0,0.347258,2172000.0,▼,1
1,20250131,6731,ピクセラ,None,3650,電気機器,9,電機・精密,-,-,2025-02-21 00:00:00+09:00,50.0,37.0,0.298851,19818500.0,▼,2
2,20250131,3928,マイネット,None,5250,情報・通信業,10,情報通信・サービスその他,-,-,2025-02-21 00:00:00+09:00,313.0,244.0,0.247756,5176600.0,▼,3
3,20250131,9162,ブリーチ,None,9050,サービス業,10,情報通信・サービスその他,-,-,2025-02-21 00:00:00+09:00,458.0,363.0,0.231425,1259700.0,△,4
4,20250131,5535,ミガロホールディングス,None,8050,不動産業,17,不動産,-,-,2025-02-21 00:00:00+09:00,7230.0,5760.0,0.226328,740500.0,▼,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3805,20250131,4629,大伸化学,None,3200,化学,4,素材・化学,-,-,2025-02-21 00:00:00+09:00,1380.0,1380.0,0.000000,1000.0,-,3806
3806,20250131,6824,新コスモス電機,None,3650,電気機器,9,電機・精密,-,-,2025-02-21 00:00:00+09:00,2790.0,2790.0,0.000000,200.0,-,3807
3807,20250131,7807,幸和製作所,None,3800,その他製品,10,情報通信・サービスその他,-,-,2025-02-21 00:00:00+09:00,859.0,859.0,0.000000,1900.0,-,3808
3808,20250131,9927,ワットマン,None,6100,小売業,14,小売,-,-,2025-02-21 00:00:00+09:00,810.0,810.0,0.000000,0.0,-,3809


In [20]:
list_header = ['#', 'コード', '銘柄名', '市場・商品区分', '33業種区分', '高値', '安値', '変化率', '出来高', '増減']
df_result = df_target[list_header].copy()
df_result.head(20)

,#,コード,銘柄名,市場・商品区分,33業種区分,高値,安値,変化率,出来高,増減
0,1,3559,ピーバンドットコム,None,卸売業,899.0,633.0,0.347258,2172000.0,▼
1,2,6731,ピクセラ,None,電気機器,50.0,37.0,0.298851,19818500.0,▼
2,3,3928,マイネット,None,情報・通信業,313.0,244.0,0.247756,5176600.0,▼
3,4,9162,ブリーチ,None,サービス業,458.0,363.0,0.231425,1259700.0,△
4,5,5535,ミガロホールディングス,None,不動産業,7230.0,5760.0,0.226328,740500.0,▼
5,6,3823,THE WHY HOW DO COMPANY,None,情報・通信業,83.0,67.0,0.213333,31183700.0,▼
6,7,3195,ジェネレーションパス,None,小売業,468.0,381.0,0.204947,1735200.0,△
7,8,5243,note,None,情報・通信業,1626.0,1327.0,0.202506,6124900.0,△
8,9,5287,イトーヨーギョー,None,ガラス・土石製品,857.0,701.0,0.200257,1427600.0,▼
9,10,5597,ブルーイノベーション,None,情報・通信業,1080.0,885.0,0.198473,2773000.0,△


In [21]:
list_col_format = ['int', 'str', 'str', 'str', 'str', 'int', 'int', 'float', 'int', 'str']
list_html = df_to_html(df_result.iloc[0:50], list_col_format)

file_html = 'up_down_ratio_%04d-%02d-%02d.html' % (dt[0].year, dt[0].month, dt[0].day)
with open(file_html, mode='w') as f:
    f.writelines(list_html)